![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/nlu/blob/master/examples/colab/Training/multi_class_text_classification/NLU_training_multi_class_text_classifier_demo_amazon.ipynb)



# Training a Deep Learning Classifier with NLU 
## ClassifierDL (Multi-class Text Classification)
## 3 class Amazon Phone review classifier training]
With the [ClassifierDL model](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl-multi-class-text-classification) from Spark NLP you can achieve State Of the Art results on any multi class text classification problem 

This notebook showcases the following features : 

- How to train the deep learning classifier
- How to store a pipeline to disk
- How to load the pipeline from disk (Enables NLU offline mode)




# 1. Install Java 8 and NLU

In [ ]:
import os
from sklearn.metrics import classification_report
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install  pyspark==2.4.7 
! pip install nlu > /dev/null    



import nlu

# 2. Download Amazon Unlocked mobile phones dataset 
https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones

dataset with unlocked mobile phone reviews in 5 review classes


In [ ]:
! wget http://ckl-it.de/wp-content/uploads/2021/01/Amazon_Unlocked_Mobile.csv

--2021-01-16 08:58:27--  http://ckl-it.de/wp-content/uploads/2021/01/Amazon_Unlocked_Mobile.csv
Resolving ckl-it.de (ckl-it.de)... 217.160.0.108, 2001:8d8:100f:f000::209
Connecting to ckl-it.de (ckl-it.de)|217.160.0.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 452621 (442K) [text/csv]
Saving to: ‘Amazon_Unlocked_Mobile.csv’

Amazon_Unlocked_Mob 100%[===================>] 442.01K   308KB/s    in 1.4s    

2021-01-16 08:58:29 (308 KB/s) - ‘Amazon_Unlocked_Mobile.csv’ saved [452621/452621]



In [ ]:
import pandas as pd
test_path = '/content/Amazon_Unlocked_Mobile.csv'
train_df = pd.read_csv(test_path,sep=",")
cols = ["y","text"]
train_df = train_df[cols]
train_df



,y,text
0,poor,"Bought it, turned it on, did not work. Opened ..."
1,average,"Currently it is 2014, the 3gs is discontinued...."
2,good,100% recomendado
3,average,It's a good phone but if you use it to browse ...
4,average,It's nice that this phone has LTE and it funct...
...,...,...
1495,poor,Not happy with this phone. Not able to get but...
1496,good,great phablet for all general uses
1497,poor,Hate this phone had it for one day
1498,good,Great cheap phone.


# 3. Train Deep Learning Classifier using nlu.load('train.classifier')

You dataset label column should be named 'y' and the feature column with text data should be named 'text'

In [ ]:
# load a trainable pipeline by specifying the train. prefix  and fit it on a datset with label and text columns
# Since there are no

trainable_pipe = nlu.load('train.classifier')
fitted_pipe = trainable_pipe.fit(train_df.iloc[:50] )


# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:50] )
preds

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


,y,text,category,default_name_embeddings,category_confidence,sentence
origin_index,,,,,,
0,poor,"Bought it, turned it on, did not work. Opened ...",average,"[0.020834514871239662, 0.03326118737459183, -0...",0.763940,"Bought it, turned it on, did not work."
0,poor,"Bought it, turned it on, did not work. Opened ...",average,"[0.030574046075344086, -0.009678893722593784, ...",1.000000,"Opened up the back, made sure it was in right,..."
0,poor,"Bought it, turned it on, did not work. Opened ...",average,"[0.023421283811330795, 0.02294657751917839, -0...",2.000000,"It was supposed to be new, but i it was used."
0,poor,"Bought it, turned it on, did not work. Opened ...",average,"[0.06009713560342789, 0.046434734016656876, -0...",3.000000,Found scratches on cover.
1,average,"Currently it is 2014, the 3gs is discontinued....",average,"[0.04893391206860542, -0.010221654549241066, -...",0.631228,"Currently it is 2014, the 3gs is discontinued."
...,...,...,...,...,...,...
47,good,Bought for my mom! She loves it!,good,"[0.021471485495567322, -0.027823669835925102, ...",0.656713,Bought for my mom!
47,good,Bought for my mom! She loves it!,good,"[0.0001737327256705612, -0.014630521647632122,...",1.000000,She loves it!
48,good,Gave the phone as a birthday gift. My friend s...,good,"[0.03572574257850647, 0.013357092626392841, -0...",0.701626,Gave the phone as a birthday gift.


# Test the fitted pipe on new example

In [ ]:
fitted_pipe.predict("It worked perfectly .")

,category,default_name_embeddings,category_confidence,sentence
origin_index,,,,
0,average,"[0.06468033790588379, -0.040837567299604416, -...",0.460187,Bitcoin is going to the moon!


## Configure pipe training parameters

In [ ]:
trainable_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['classifier_dl'] has settable params:
pipe['classifier_dl'].setMaxEpochs(3)                | Info: Maximum number of epochs to train | Currently set to : 3
pipe['classifier_dl'].setLr(0.005)                   | Info: Learning Rate | Currently set to : 0.005
pipe['classifier_dl'].setBatchSize(64)               | Info: Batch size | Currently set to : 64
pipe['classifier_dl'].setDropout(0.5)                | Info: Dropout coefficient | Currently set to : 0.5
pipe['classifier_dl'].setEnableOutputLogs(True)      | Info: Whether to use stdout in addition to Spark logs. | Currently set to : True
>>> pipe['default_tokenizer'] has settable params:
pipe['default_tokenizer'].setTargetPattern('\S+')    | Info: pattern to grab from text as token candidates. Defaults \S+ | Currently set to : \S+
pipe['default_tokenizer'].setContextChars(['.', ',', ';', ':', '!', '?', '*', '-', '(', ')', '"', 

## Retrain with new parameters

In [ ]:
# Train longer!
trainable_pipe['classifier_dl'].setMaxEpochs(5)  
fitted_pipe = trainable_pipe.fit(train_df.iloc[:100])
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:100],output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['category']))
preds

              precision    recall  f1-score   support

     average       0.00      0.00      0.00        29
        good       0.65      0.94      0.77        32
        poor       0.69      0.95      0.80        39

    accuracy                           0.67       100
   macro avg       0.45      0.63      0.52       100
weighted avg       0.48      0.67      0.56       100



,y,text,document,category,default_name_embeddings,category_confidence
origin_index,,,,,,
0,poor,"Bought it, turned it on, did not work. Opened ...","Bought it, turned it on, did not work. Opened ...",poor,"[0.059367865324020386, 0.05043933913111687, -0...",0.952295
1,average,"Currently it is 2014, the 3gs is discontinued....","Currently it is 2014, the 3gs is discontinued....",good,"[0.0046275281347334385, 0.012452688068151474, ...",0.396265
2,good,100% recomendado,100% recomendado,good,"[0.008266163989901543, 0.00396152026951313, -0...",0.773682
3,average,It's a good phone but if you use it to browse ...,It's a good phone but if you use it to browse ...,poor,"[0.05291805788874626, 0.002292224671691656, -0...",0.506015
4,average,It's nice that this phone has LTE and it funct...,It's nice that this phone has LTE and it funct...,good,"[0.03426238149404526, -0.024366019293665886, -...",0.648859
...,...,...,...,...,...,...
95,poor,"Hola, compramos dos telÃ©fonos y vienieron tot...","Hola, compramos dos telÃ©fonos y vienieron tot...",poor,"[0.06324272602796555, -0.06387951225042343, -0...",0.790492
96,good,Excelente,Excelente,good,"[0.03246314451098442, -0.01719777286052704, -0...",0.813424
97,poor,the product is good but the English language s...,the product is good but the English language s...,poor,"[0.056343767791986465, -0.016822000965476036, ...",0.940151


# Try training with different Embeddings

In [ ]:
# We can use nlu.print_components(action='embed_sentence') to see every possibler sentence embedding we could use. Lets use bert!
nlu.print_components(action='embed_sentence')

For language <en> NLU provides the following Models : 
nlu.load('en.embed_sentence') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.tfhub_use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.tfhub_use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.albert') returns Spark NLP model albert_base_uncased
nlu.load('en.embed_sentence.electra') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_small_uncased') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_base_uncased') returns Spark NLP model sent_electra_base_uncased
nlu.load('en.embed_sentence.electra_large_uncased') returns Spark NLP model sent_electra_large_uncased
nlu.load('en.embed_sentence.bert') returns Spark NLP model sent_bert_base_uncased
nlu.load('en.embed_sentenc

In [ ]:
from sklearn.metrics import classification_report
trainable_pipe = nlu.load('en.embed_sentence.small_bert_L12_768 train.classifier')
# We need to train longer and user smaller LR for NON-USE based sentence embeddings usually
# We could tune the hyperparameters further with hyperparameter tuning methods like gridsearch
# Also longer training gives more accuracy
trainable_pipe['classifier_dl'].setMaxEpochs(90)  
trainable_pipe['classifier_dl'].setLr(0.0005) 
fitted_pipe = trainable_pipe.fit(train_df)
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df,output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['category']))

#preds

sent_small_bert_L12_768 download started this may take some time.
Approximate size to download 392.9 MB
[OK!]
              precision    recall  f1-score   support

     average       0.72      0.67      0.69       500
        good       0.85      0.87      0.86       500
        poor       0.78      0.83      0.80       500

    accuracy                           0.79      1500
   macro avg       0.78      0.79      0.79      1500
weighted avg       0.78      0.79      0.79      1500



# 5. Lets save the model

In [ ]:
stored_model_path = './models/classifier_dl_trained' 
fitted_pipe.save(stored_model_path)

Stored model in ./models/classifier_dl_trained


# 6. Lets load the model from HDD.
This makes Offlien NLU usage possible!   
You need to call nlu.load(path=path_to_the_pipe) to load a model/pipeline from disk.

In [ ]:
hdd_pipe = nlu.load(path=stored_model_path)

preds = hdd_pipe.predict('It worked perfectly.')
preds

,classifier_confidence,document,en_embed_sentence_small_bert_L12_768_embeddings,classifier
origin_index,,,,
0,0.950214,It worked perfectly.,"[0.275971919298172, 0.4924655854701996, 0.2755...",good


In [ ]:
hdd_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['document_assembler'] has settable params:
pipe['document_assembler'].setCleanupMode('shrink')             | Info: possible values: disabled, inplace, inplace_full, shrink, shrink_full, each, each_full, delete_full | Currently set to : shrink
>>> pipe['sentence_detector'] has settable params:
pipe['sentence_detector'].setCustomBounds([])                   | Info: characters used to explicitly mark sentence bounds | Currently set to : []
pipe['sentence_detector'].setDetectLists(True)                  | Info: whether detect lists during sentence detection | Currently set to : True
pipe['sentence_detector'].setExplodeSentences(False)            | Info: whether to explode each sentence into a different row, for better parallelization. Defaults to false. | Currently set to : False
pipe['sentence_detector'].setMaxLength(99999)                   | Info: Set the maximum allowed length f